In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import validators
import datetime
import math
import re
import ipynb
import time
import warnings

In [44]:
warnings.filterwarnings("ignore")

In [45]:
def find_tables_flex(first, last, team, year):
    for i in range(0,6):
        try:
            if len(last) < 4:
                last = last + 'x'
            url = 'https://www.pro-football-reference.com/players/%s/%s%s0%i.htm' % (last[0], last[0:4], 
                                                                                     first[0:2], i)
            response = requests.get(url)
            x = pd.read_html(url)
            df = x[0]
            df[df.columns[0]] = df[df.columns[0]].str.replace(r'[\*+]', '', regex=True)
            if (df[df[df.columns[0]] == str(year)][df.columns[2]]).values == team:
                return x[0], response
        except:
            pass
    return 0, 0

def find_age(response, year):
    soup = BeautifulSoup(response.content, "html.parser")
    x = soup.find('div', {'id': 'info'})
    dob = x.find('span"', {'id': 'necro-birth'})['data-birth']
    return math.floor(((datetime.date(year, 9, 7) - datetime.date(int(dob[0:4]), int(dob[5:7]), int(dob[8:10]))).days)/365)

In [46]:
tes = pd.read_csv('tes.csv')

In [47]:
tes.drop(columns = 'Unnamed: 0', inplace = True)

In [48]:
te2022 = pd.read_csv('2022tes.csv')
corr_teams = pd.read_html('https://www.pro-football-reference.com/years/2022/fantasy.htm')
corrte2022 = corr_teams[0][[('Unnamed: 1_level_0',  'Player'), ('Unnamed: 2_level_0',      'Tm'), ('Unnamed: 3_level_0', 'FantPos')]]

In [49]:
corrte2022[('Unnamed: 1_level_0',  'Player')] = corrte2022[('Unnamed: 1_level_0',  'Player')].str.replace(r'[\*+]', '', 
                                                                                                regex=True)

In [50]:
corrte2022 = corrte2022[corrte2022[('Unnamed: 3_level_0', 'FantPos')] == 'TE']

In [51]:
te2022['Player'] = te2022['Player'].replace('Jr.', '', regex = True)
te2022['Player'] = te2022['Player'].replace('Shenker', '', regex = True)

In [52]:
x = te2022['Player'].str.split(' ', expand = True)

In [53]:
for i in range(0, len(x)):
    if x.loc[i,2] == "":
        x.loc[i,2] = x.loc[i,3]
x.drop(columns = 3, inplace = True)
x[2] = x[2].replace(r'[()]', '', regex = True)

In [54]:
te2022[['First', 'Last', 'Tm']] = x
te2022['Player'] = [i.strip() if isinstance(i,str) else i for i in te2022['Player'].str.split('(', expand = True)[0]]
te2022['Year'] = 2022
te2022['Pos'] = 'TE'
te2022.dropna(inplace = True)
te2022['Age'] = None
te2022.drop(columns = 'Rank', inplace = True)

In [55]:
x = pd.DataFrame()
x['Player'] = corrte2022[('Unnamed: 1_level_0',  'Player')]
x['Tm'] = corrte2022[('Unnamed: 2_level_0',      'Tm')]
x['Pos'] = corrte2022[('Unnamed: 3_level_0', 'FantPos')]

In [56]:
te2022 = pd.merge(te2022, x, on = 'Player')

In [57]:
te2022.drop(columns = ['Tm_x', 'Pos_x'], inplace = True)
te2022.rename(columns = {'Tm_y': 'Tm', 'Pos_y': 'Pos'}, inplace = True)

In [58]:
te2022

,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,...,G,FPTS,FPTS/G,ROST,First,Last,Year,Age,Tm,Pos
0,Travis Kelce,110.0,152.0,"1,338",12.2,52.0,29.0,12.0,2.0,5.0,...,17.0,316.3,18.6,100.0%,Travis,Kelce,2022,None,KAN,TE
1,T.J. Hockenson,86.0,129.0,914,10.6,81.0,12.0,6.0,0.0,0.0,...,17.0,215.4,12.7,99.3%,T.J.,Hockenson,2022,None,2TM,TE
2,George Kittle,60.0,86.0,765,12.8,54.0,20.0,11.0,0.0,0.0,...,15.0,200.5,13.4,99.1%,George,Kittle,2022,None,SFO,TE
3,Mark Andrews,73.0,113.0,847,11.6,36.0,8.0,5.0,3.0,8.0,...,15.0,190.5,12.7,99.9%,Mark,Andrews,2022,None,BAL,TE
4,Evan Engram,73.0,98.0,766,10.5,36.0,11.0,4.0,2.0,13.0,...,17.0,176.9,10.4,94.3%,Evan,Engram,2022,None,JAX,TE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Tommy Sweeney,1.0,1.0,7,7.0,7.0,0.0,0.0,0.0,0.0,...,2.0,1.7,0.9,0.0%,Tommy,Sweeney,2022,None,BUF,TE
110,Jacob Harris,1.0,1.0,6,6.0,6.0,0.0,0.0,0.0,0.0,...,3.0,1.6,0.5,0.0%,Jacob,Harris,2022,None,LAR,TE
111,Richard Rodgers,1.0,4.0,4,4.0,4.0,0.0,0.0,0.0,0.0,...,3.0,1.4,0.5,0.0%,Richard,Rodgers,2022,None,LAC,TE
112,Feleipe Franks,0.0,2.0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,0.0,0.0,0.0%,Feleipe,Franks,2022,None,ATL,TE


In [59]:
column_order = ['First', 'Last', 'Year', 'Tm', 'Pos', 'Age', 'FPTS', 'Player']
te2022 = te2022.reindex(columns = column_order)
te2022.rename(columns = {'FPTS': 'FantasyPoints'}, inplace = True)

In [60]:
totaltes = pd.concat([tes, te2022])
totaltes = totaltes.reset_index().drop(columns = 'index')

In [61]:
totaltes = totaltes.sort_values('FantasyPoints', ascending = False).groupby('Year').head(50)

In [62]:
totaltes['First'] = totaltes['First'].replace('Delanie', 'Hubert', regex = True)
totaltes['First'] = totaltes['First'].replace('T.J.', 'TJ', regex = True)
totaltes['First'] = totaltes['First'].replace("'", '', regex = True)
totaltes['Last'] = totaltes['Last'].replace("'", '', regex = True)

In [63]:
totaltes = totaltes.reset_index().drop(columns = 'index')

In [64]:
totaltes['Targets'] = np.nan
totaltes['Receptions'] = np.nan
totaltes['RecYards'] = np.nan
totaltes['RecTD'] = np.nan
totaltes['Ctch%'] = np.nan
totaltes['RushAtt'] = np.nan
totaltes['RushYards'] = np.nan
totaltes['RushTD'] = np.nan
totaltes['Fumbles'] = np.nan
totaltes['Experience'] = np.nan

In [67]:
totaltes[totaltes['Player'] == 'T.J. Hockenson']

,First,Last,Year,Tm,Pos,Age,FantasyPoints,Player,Targets,Receptions,RecYards,RecTD,Ctch%,RushAtt,RushYards,RushTD,Fumbles,Experience
44,TJ,Hockenson,2022,2TM,TE,NaN,215.4,T.J. Hockenson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,TJ,Hockenson,2020,DET,TE,23.0,175.0,T.J. Hockenson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,TJ,Hockenson,2021,DET,TE,24.0,145.0,T.J. Hockenson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478,TJ,Hockenson,2019,DET,TE,22.0,80.7,T.J. Hockenson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
start = time.time()
for i in range(44,45):
    if i % 100 == 0:
        print(i, totaltes.loc[i, 'First'],totaltes.loc[i, 'Last'], time.time() - start)
    try:    
        stats, response = find_tables_flex(totaltes.loc[i, 'First'],totaltes.loc[i, 'Last'], 
                                       totaltes.loc[i, 'Tm'], totaltes.loc[i, 'Year'])
        stats[(( 'Unnamed: 0_level_0',  'Year'))] = pd.to_numeric(stats[(( 'Unnamed: 0_level_0',  'Year'))], errors = 'coerce')
        
        if totaltes.loc[i, 'Tm'] == '2TM':
            idx = stats[(stats[( 'Unnamed: 2_level_0', 'Tm')] == '2TM') & (stats[( 'Unnamed: 0_level_0', 'Year')] == totaltes.loc[i, 'Year'])].index + 1
            totaltes.loc[i, 'Tm'] = stats.loc[idx, ( 'Unnamed: 2_level_0',    'Tm')].iloc[0]
        stats.dropna(subset = [(( 'Unnamed: 1_level_0',   'Age'))], inplace = True)
        
        totaltes.loc[i, 'Experience'] = len(stats[stats[(( 'Unnamed: 0_level_0',  'Year'))] <= totaltes.loc[i, 'Year']])
        
        if totaltes.loc[i, 'Year'] == 2022:
            totaltes.loc[i, 'Age'] = find_age(response, totaltes.loc[i, 'Year'])

        date = stats[(stats[(( 'Unnamed: 0_level_0',  'Year'))] < (totaltes.loc[i, 'Year'])) & (stats[(( 'Unnamed: 0_level_0',  'Year'))] >= (totaltes.loc[i, 'Year'] - 2))].reset_index()
        date[('Receiving', 'Ctch%')] = date[('Receiving', 'Ctch%')].replace('%', '', regex = True)
        date.drop(columns = [( 'Unnamed: 2_level_0',     'Tm'), 
                             ( 'Unnamed: 3_level_0',     'Pos')], inplace = True)
        if ('Unnamed: 32_level_0', 'Awards') in date.columns:
            date.drop(columns = [('Unnamed: 32_level_0', 'Awards')], inplace = True)
        date = date.astype(float)
        
        totaltes.loc[i, 'Targets'] = (date[('Receiving', 'Tgt')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totaltes.loc[i, 'Receptions'] = (date[('Receiving', 'Rec')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totaltes.loc[i, 'RecYards'] = (date[('Receiving', 'Yds')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totaltes.loc[i, 'RecTD'] = (date[('Receiving', 'TD')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totaltes.loc[i, 'Ctch%'] = (date[('Receiving', 'Ctch%')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totaltes.loc[i, 'RushAtt'] = (date[('Rushing', 'Att')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totaltes.loc[i, 'RushYards'] = (date[('Rushing', 'Yds')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
        totaltes.loc[i, 'RushTD'] = (date[('Rushing', 'TD')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games','G')]*(date.index+1)).sum()
        totaltes.loc[i, 'Fumbles'] = (date[('Unnamed: 30_level_0', 'Fmb')]*date[('Games', 'G')]*(date.index+1)).sum()/(date[('Games', 'G')]*(date.index+1)).sum()
    except:
        print("Error:", totaltes.loc[i, 'First'], totaltes.loc[i, 'Last'], totaltes.loc[i, 'Tm'], totaltes.loc[i, 'Year'])
    time.sleep(12)
print(time.time() - start)

12.465520858764648


In [683]:
finaltes = totaltes.dropna()

In [687]:
finaltes.to_csv('finaltes.csv')